In [ ]:
%%capture

!pip install gdown gpxpy contextily

In [ ]:
%%capture

import gdown

FOLDER_URL = 'https://drive.google.com/drive/folders/1SOqJ2zJtu4qYgT0_UeQrOgFWGHxMeDe_'
gdown.download_folder(FOLDER_URL, quiet=True, use_cookies=False)

In [ ]:
import pandas as pd

constellation_names = {
    0: 'Unknown',
    1: 'GPS',
    3: 'Glonass',
    4: 'QZSS',
    5: 'Beidou',
    6: 'Galileo',
}

gnss_logger_df = pd.read_csv('/content/polar-vs-gnss-logger/status_gnss_log_2024_11_28_04_57_56.txt')

gnss_logger_df['Time'] = pd.to_datetime(gnss_logger_df['UnixTimeMillis'], unit='ms', utc=True)
gnss_logger_df['ConstellationName'] = gnss_logger_df['ConstellationType'].map(constellation_names).astype(str)

gnss_logger_df.head()

In [ ]:
gnss_logger_df.columns

In [ ]:
import gpxpy
import pandas as pd

def parse_gpx(filepath):
  gpx_file = open(filepath, 'r')
  gpx = gpxpy.parse(gpx_file)

  data = []

  for track in gpx.tracks:
    for segment in track.segments:
      for point in segment.points:
        time, lat, lng = point.time, point.latitude, point.longitude,
        data.append({
          'time': time,
          'lat': lat,
          'lon': lng,
        })

  df = pd.DataFrame(data)
  df['time'] = pd.to_datetime(df['time'])

  return df

polar_df = parse_gpx('/content/polar-vs-gnss-logger/Evgeny_Arbatov_2024-11-28_04-59-24.GPX')
polar_df.head()

In [ ]:
osmtracker_df = parse_gpx('/content/polar-vs-gnss-logger/osm-upload1772647869123905882.gpx')
osmtracker_df.head()

In [ ]:
import matplotlib.pyplot as plt
import contextily as ctx

def plot_gpx(df):
  fig, ax = plt.subplots(figsize=(10, 10))
  ax.plot(df['lon'], df['lat'], 'r-', label="Route")
  ctx.add_basemap(ax, crs='EPSG:4326', source=ctx.providers.OpenStreetMap.Mapnik)
  ax.set_aspect('equal', adjustable='box')
  ax.set_xticks([])
  ax.set_yticks([])
  ax.tick_params(axis='both', which='both', bottom=False, top=False, left=False, right=False)
  plt.show()

plot_gpx(polar_df)

In [ ]:
plot_gpx(osmtracker_df)

In [ ]:
polar_start_time = polar_df.iloc[0]['time']
polar_start_time

In [ ]:
gnss_logger_polar_fix_time = gnss_logger_df[gnss_logger_df['Time'] >= polar_start_time].iloc[0]['Time']
gnss_logger_polar_fix_time

In [ ]:
polar_fix_df = gnss_logger_df[gnss_logger_df['Time'] == gnss_logger_polar_fix_time]
polar_fix_df.head()

In [ ]:
polar_fix_df.groupby('ConstellationName')['Svid'].nunique().reset_index()

In [ ]:
before_polar_fix_df = gnss_logger_df[gnss_logger_df['Time'] < gnss_logger_polar_fix_time]
before_polar_fix_df.groupby('ConstellationName')['Svid'].nunique().reset_index()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

grouped = before_polar_fix_df[before_polar_fix_df['Cn0DbHz'] >= 34].groupby(['Time', 'ConstellationName'])['Svid'].nunique().reset_index()
grouped.columns = ['Time', 'ConstellationName', 'SvidUnqiueCount']

pivoted = grouped.pivot(index='Time', columns='ConstellationName', values='SvidUnqiueCount').fillna(0)

ax = pivoted.plot(figsize=(12, 6))
ax.yaxis.set_major_locator(MaxNLocator(integer=True))
ax.grid(True, linestyle='--', alpha=0.7)
ax.set_ylabel('Satellite Count (Cn0DbHz >= 34)')
ax.set_title('Satellite Count by Constellation Name')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

gps_galileo_before_fix_df = before_polar_fix_df[before_polar_fix_df['ConstellationName'].isin(['GPS', 'Galileo'])]

gps_galileo_before_fix_df = gps_galileo_before_fix_df.set_index('Time')
grouped_gps_galileo_before_fix_df = gps_galileo_before_fix_df.groupby('ConstellationName')

fig, ax = plt.subplots(figsize=(10, 6))
for name, group in grouped_gps_galileo_before_fix_df:
    group.plot(y='Cn0DbHz', ax=ax, label=name)

ax.set_xlabel('Time')
ax.set_ylabel('Cn0DbHz')
ax.legend(title='Constellation')
plt.tight_layout()
plt.show()

In [ ]:
before_polar_fix_df[before_polar_fix_df['Cn0DbHz'] >= 35].groupby('ConstellationName')['Svid'].nunique().reset_index()

In [ ]:
gnss_logger_df[gnss_logger_df['UsedInFix'] == 1].groupby('ConstellationName')['Svid'].nunique().reset_index()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

grouped = gnss_logger_df[gnss_logger_df['UsedInFix'] == 1].groupby(['Time', 'ConstellationName'])['Svid'].nunique().reset_index()
grouped.columns = ['Time', 'ConstellationName', 'SvidUniqueCount']

pivoted = grouped.set_index('Time').groupby('ConstellationName')['SvidUniqueCount'].resample('1Min').mean().unstack(level=0).fillna(0)

fig, ax = plt.subplots(figsize=(12, 6), dpi=300)
pivoted.plot(ax=ax)
ax.yaxis.set_major_locator(MaxNLocator(integer=True))
ax.grid(True, linestyle='--', alpha=0.7)
ax.set_ylabel('Satellite Count')
ax.set_title('Count of Satellites Used in GPS Fix')
plt.legend()
plt.show()